In [1]:
import sqlite3
import pandas as pd
import csv
import numpy as np

In [25]:
conn = sqlite3.connect('fin_db.db')
cursor = conn.cursor()


In [26]:
# cursor.execute('''DROP TABLE fin_db
#                 ''')

In [27]:
cursor.execute('''CREATE TABLE IF NOT EXISTS fin_db (
                    id INTEGER,
                    text TEXT,
                    embeding INTEGER
                )''')

In [28]:
df = pd.read_excel('../../out1.xlsx', names = ['Sentences', 'Embbeding'])

In [29]:
df['index'] = df.index + 1

In [30]:
df[['index','Embbeding', 'Sentences']].to_csv('out1.csv', index = False)

In [31]:
with open('out1.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # Пропуск заголовков столбцов
    cursor.executemany('INSERT INTO fin_db VALUES (?, ?, ?)', csv_data)

In [35]:
conn.commit()
conn.close()

In [59]:
conn = sqlite3.connect('fin_db.db')
cursor = conn.cursor()

In [60]:
cursor.execute("SELECT * FROM fin_db")
rows = cursor.fetchall()

OperationalError: no such table: fin_db

In [39]:
input_vector = [float(x) for x in rows[0][1].split(',')]

In [40]:
# vectors = [np.frombuffer(vector[0]) for vector in vectors]

In [41]:
from scipy.spatial.distance import cosine

In [50]:
# Расчет косинусного расстояния и сохранение результатов
distances = []
for row in rows:
    vector = [float(x) for x in row[1].split(',')]
    distance = cosine(input_vector, vector)
    distances.append((distance, row[1], row[0],row[2]))  # Замените на соответствующий столбец с другими данными

# Сортировка результатов по возрастанию расстояния
distances.sort(key=lambda x: x[0])

# Вывод 10 самых близких расстояний и соответствующих данных
# for i in range(10):
#     print(f"Расстояние: {distances[i][0]}, Другие данные: {distances[i][1]}")

# Закрытие соединения с базой данных
conn.close()

In [57]:
number_sent = 2
out = []
ids = []
for i in range(number_sent):
    out.append(distances[i][3])
    ids.append(distances[i][2])

In [58]:
ids

[1, 49]

In [44]:
out = []
for i in range(10):
    out.append(distances[i][2])

In [45]:
out

[1, 49, 4, 26, 39, 22, 29, 50, 48, 51]

In [ ]:
После создания базы данных, вы можете использовать ее внутри Docker контейнера. Для этого вам нужно создать Dockerfile, который будет содержать инструкции для создания контейнера. Вот пример Dockerfile:

```Dockerfile
FROM python:3.9

# Копирование файлов в контейнер
COPY database.db /app/database.db
COPY script.py /app/script.py
COPY data.csv /app/data.csv

# Установка зависимостей
RUN pip install sqlite3

# Запуск скрипта при запуске контейнера
CMD python /app/script.py
```

Затем, вы можете собрать Docker образ и запустить контейнер с помощью следующих команд:

```
docker build -t my_app .
docker run my_app
```

При этом, файлы `database.db`, `script.py` и `data.csv` должны находиться в той же директории, где находится Dockerfile.